<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w4_model_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training & Evaluation

## ##Training & Evaluation is currently being carried out in google Colaboratory due to less free resources available in IBM watson studio

# Download Data

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
'''
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')
'''

In [3]:
#download_file_cos(credentials_2,'train_feat_eng.csv','train_feat_eng.csv')
#ownload_file_cos(credentials_2,'test_feat_eng.csv','test_feat_eng.csv')
#download_file_cos(credentials_2,'sample_submission.csv','sample_submission.csv')
!ls

connectivity_model_nn_03.h5
data.covs.pickle
dipole_moments.csv.zip
kaggle.json
magnetic_shielding_tensors.csv.zip
mulliken_charges.csv.zip
pairs_dict.pkl
potential_energy.csv.zip
sample_data
sample_submission.csv.zip
scalar_coupling_contributions.csv.zip
structures.csv.zip
structures.zip
test.csv.zip
test_etl.csv
test_feat_eng_v2_orig_my_ext_not_std.csv
test_feat_eng_v2_orig_my_ext_std.csv
test_feat_eng_v2_orig_my_not_std.csv
test_feat_eng_v2_orig_my_std.csv
train.csv.zip
train_etl.csv
train_feat_eng_v2_orig_my_ext_not_std.csv
train_feat_eng_v2_orig_my_ext_std.csv
train_feat_eng_v2_orig_my_not_std.csv
train_feat_eng_v2_orig_my_std.csv


# EXP NO.1 (orig+my+std)

# Dowload Data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
%matplotlib inline

In [2]:
tf.__version__

'1.14.0'

In [4]:
train=pd.read_csv('train_feat_eng_v2_orig_my_std.csv')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,-0.058940,0.101017,-0.028861,C,-0.074536,0.838296,-0.055483,0.263942,-2.057703,1JHC,84.8076,-1.722482,0,0,1,c_h,2,0.919759,1
1,1,dsgdb9nsd_000001,1,2,H,-0.058940,0.101017,-0.028861,H,0.617919,1.052165,-0.061633,0.263942,0.729230,2JHH,-11.2570,-0.784301,3,1,1,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,-0.058940,0.101017,-0.028861,H,-0.431513,1.042984,-0.759868,0.263942,0.729234,2JHH,-11.2548,-0.784264,3,1,1,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,-0.058940,0.101017,-0.028861,H,-0.420021,1.037555,0.659852,0.263942,0.729234,2JHH,-11.2543,-0.784251,3,1,1,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,0.514726,0.793727,-0.029947,C,-0.074536,0.838296,-0.055483,0.263964,-2.057703,1JHC,84.8074,-1.722484,0,0,1,c_h,2,0.919759,1


In [5]:
train.shape

(4658147, 24)

In [6]:
test=pd.read_csv('test_feat_eng_v2_orig_my_std.csv')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.004343,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.004343,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.004343,0.10386,0.609053,H,1.057219,0.223874,0.734382,-7.526702e-13,0.663743,3JHH,1.306289,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,0.884018,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,0.884018,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0


In [7]:
test.shape

(2505542, 23)

# Drop Columns

In [8]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int'],
      dtype='object')

In [0]:
drop_cols=['id', 'molecule_name','atom_0','atom_1','type', 'pairs','bond']

In [10]:
train.drop(drop_cols,axis=1,inplace=True)
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs_enc,bond_int
0,1,0,-0.058940,0.101017,-0.028861,-0.074536,0.838296,-0.055483,0.263942,-2.057703,84.8076,-1.722482,0,0,1,2,1
1,1,2,-0.058940,0.101017,-0.028861,0.617919,1.052165,-0.061633,0.263942,0.729230,-11.2570,-0.784301,3,1,1,11,1
2,1,3,-0.058940,0.101017,-0.028861,-0.431513,1.042984,-0.759868,0.263942,0.729234,-11.2548,-0.784264,3,1,1,11,1
3,1,4,-0.058940,0.101017,-0.028861,-0.420021,1.037555,0.659852,0.263942,0.729234,-11.2543,-0.784251,3,1,1,11,1
4,2,0,0.514726,0.793727,-0.029947,-0.074536,0.838296,-0.055483,0.263964,-2.057703,84.8074,-1.722484,0,0,1,2,1


In [11]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'distance', 'type_enc', 'atom_1_enc',
       'atom_0_enc', 'pairs', 'pairs_enc', 'bond', 'bond_int'],
      dtype='object')

In [12]:
test.drop(drop_cols,axis=1,inplace=True)
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc,atom_1_enc,atom_0_enc,pairs_enc,bond_int
0,2,0,-1.004343,0.10386,0.609053,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,-0.135391,2,0,1,2,0
1,2,1,-1.004343,0.10386,0.609053,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,-1.763005,0,0,1,2,1
2,2,3,-1.004343,0.10386,0.609053,1.057219,0.223874,0.734382,-7.526702e-13,0.663743,1.306289,6,1,1,11,0
3,3,0,0.884018,0.10386,0.609053,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,-1.763005,0,0,1,2,1
4,3,1,0.884018,0.10386,0.609053,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,-0.135391,2,0,1,2,0


In [13]:
train.dtypes

atom_index_0                  int64
atom_index_1                  int64
x_0                         float64
y_0                         float64
z_0                         float64
x_1                         float64
y_1                         float64
z_1                         float64
mulliken_charge_0           float64
mulliken_charge_1           float64
scalar_coupling_constant    float64
distance                    float64
type_enc                      int64
atom_1_enc                    int64
atom_0_enc                    int64
pairs_enc                     int64
bond_int                      int64
dtype: object

In [14]:
test.dtypes

atom_index_0           int64
atom_index_1           int64
x_0                  float64
y_0                  float64
z_0                  float64
x_1                  float64
y_1                  float64
z_1                  float64
mulliken_charge_0    float64
mulliken_charge_1    float64
distance             float64
type_enc               int64
atom_1_enc             int64
atom_0_enc             int64
pairs_enc              int64
bond_int               int64
dtype: object

In [15]:
print(train.shape)
print(test.shape)

(4658147, 17)
(2505542, 16)


# Creating training, validation and testing data

In [16]:
x=train.drop('scalar_coupling_constant',axis=1)
x.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc,atom_1_enc,atom_0_enc,pairs_enc,bond_int
0,1,0,-0.058940,0.101017,-0.028861,-0.074536,0.838296,-0.055483,0.263942,-2.057703,-1.722482,0,0,1,2,1
1,1,2,-0.058940,0.101017,-0.028861,0.617919,1.052165,-0.061633,0.263942,0.729230,-0.784301,3,1,1,11,1
2,1,3,-0.058940,0.101017,-0.028861,-0.431513,1.042984,-0.759868,0.263942,0.729234,-0.784264,3,1,1,11,1
3,1,4,-0.058940,0.101017,-0.028861,-0.420021,1.037555,0.659852,0.263942,0.729234,-0.784251,3,1,1,11,1
4,2,0,0.514726,0.793727,-0.029947,-0.074536,0.838296,-0.055483,0.263964,-2.057703,-1.722484,0,0,1,2,1


In [17]:
y=train['scalar_coupling_constant']
y.head()

0    84.8076
1   -11.2570
2   -11.2548
3   -11.2543
4    84.8074
Name: scalar_coupling_constant, dtype: float64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3)

In [20]:
print('x_train: {}   y_train: {}'.format(x_train.shape,y_train.shape))
print('x_val: {}   y_val: {}'.format(x_val.shape,y_val.shape))

x_train: (3260702, 16)   y_train: (3260702,)
x_val: (1397445, 16)   y_val: (1397445,)


In [21]:
x_test=test
x_test.shape

(2505542, 16)

# Training & Evaluation

In [22]:
import gc
gc.collect()

0

## Linear Model (Baseline)

In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
model_lr=LinearRegression()

In [26]:
model_lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
model_lr.score(x_train,y_train)

0.6465675917835193

In [0]:
from sklearn.metrics import mean_absolute_error

In [0]:
y_pred=model_lr.predict(x_val)

In [32]:
mean_absolute_error(y_val,y_pred)

16.263165786971904

## GBM (Baseline)

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
model_gbm=GradientBoostingRegressor()

In [35]:
model_gbm.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [36]:
model_gbm.score(x_train,y_train)

0.988358235128943

In [0]:
y_pred=model_gbm.predict(x_val)

In [40]:
mean_absolute_error(y_val,y_pred)

2.4829541034621534

## lightgbm (Baseline test)

In [0]:
from lightgbm import LGBMRegressor
import lightgbm

In [0]:
%%timeit
params = {'boosting': 'gbdt', 'colsample_bytree': 1, 
          'learning_rate': 0.1, 'max_depth': 40, 'metric': 'mae',
          'min_child_samples': 50, 'num_leaves': 500, 
          'objective': 'regression', 'reg_alpha': 0.5, 
          'reg_lambda': 0.8, 'subsample': 0.5 }
lgtrain = lightgbm.Dataset(x_train, label=y_train)
lgval = lightgbm.Dataset(x_val, label=y_val)

model_lgb = lightgbm.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval], early_stopping_rounds=5, 
                      verbose_eval=500)


Training until validation scores don't improve for 5 rounds.


## NN Model

In [0]:
input_shape=(38,)
model_nn=keras.Sequential()
model_nn.add(keras.layers.Dense(64,activation='relu',input_shape=input_shape))
model_nn.add(keras.layers.Dense(128,activation='relu'))
#model_nn.add(keras.layers.Dense(256,activation='relu'))
#model_nn.add(keras.layers.Dense(512,activation='relu'))
model_nn.add(keras.layers.Dense(128,activation='relu'))
model_nn.add(keras.layers.Dense(64,activation='relu'))
model_nn.add(keras.layers.Dense(8,activation='relu'))
model_nn.add(keras.layers.Dense(1))

model_nn.summary()

W0814 16:29:07.790420 139934774769536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2496      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 36,113
Trainable params: 36,113
Non-trainable params: 0
____________________________________________________

In [0]:
model_nn.compile(optimizer='adam',loss='mean_absolute_error')

In [0]:
model_nn.fit(x_train,y_train,epochs=20,batch_size=512,validation_data=(x_val,y_val))

Train on 3260702 samples, validate on 1397445 samples
Epoch 1/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.9214 - val_loss: 3.6140
Epoch 2/20
3260702/3260702 [==============================] - 31s 9us/sample - loss: 3.6764 - val_loss: 3.5721
Epoch 3/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.6493 - val_loss: 3.5280
Epoch 4/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.4700 - val_loss: 3.4285
Epoch 5/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.3946 - val_loss: 3.3059
Epoch 6/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.3506 - val_loss: 3.2760
Epoch 7/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.3378 - val_loss: 3.2740
Epoch 8/20
3260702/3260702 [==============================] - 30s 9us/sample - loss: 3.2873 - val_loss: 3.2224
Epoch 9/20
3260702/3260702 [==============================

# Train on whole data with the best model

In [0]:
model_nn.fit(x,y,epochs=5,batch_size=512,validation_data=(x_val,y_val))

Train on 4658147 samples, validate on 1397445 samples
Epoch 1/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.1120 - val_loss: 3.0486
Epoch 2/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.0955 - val_loss: 3.1948
Epoch 3/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.0829 - val_loss: 3.0855
Epoch 4/5
4658147/4658147 [==============================] - 41s 9us/sample - loss: 3.0748 - val_loss: 2.9660
Epoch 5/5
4658147/4658147 [==============================] - 40s 9us/sample - loss: 3.0661 - val_loss: 3.0940


# Serialize the model

In [0]:
tf.keras.models.save_model(model_nn,'kaggle_model_nn_v2_with_bonds.h5',overwrite=True,include_optimizer=True)

In [0]:
!ls

# Create Submission file

In [0]:
import gc
gc.collect()

634

In [0]:
#test=pd.read_csv('test_feat_eng_v2.csv')
test_predictions=model_nn.predict(test,verbose=1,batch_size=10000)
test_predictions[:10]

2505542/2505542 [==============================] - 1s 1us/sample


array([[ 20.150017 ],
       [  0.5381685],
       [ -3.6198368],
       [  0.5381685],
       [  4.5417128],
       [ 74.119545 ],
       [ -9.192482 ],
       [ -5.4883533],
       [-11.815308 ],
       [ 75.20229  ]], dtype=float32)

In [0]:
test_predictions.shape

(2505542, 1)

In [0]:
sample_submission=pd.read_csv('sample_submission.csv.zip')
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [0]:
sample_submission['scalar_coupling_constant']=test_predictions
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,20.150017
1,4658148,0.538168
2,4658149,-3.619837
3,4658150,0.538168
4,4658151,4.541713


In [0]:
sample_submission.to_csv('sample_submission_external_feat_v2_with_bonds.csv',index=False)

In [0]:
!ls

data.covs.pickle
dipole_moments.csv.zip
kaggle.json
kaggle_model_nn_v2_04.h5
magnetic_shielding_tensors.csv.zip
mulliken_charges.csv.zip
potential_energy.csv.zip
sample_data
sample_submission.csv.zip
sample_submission_distance_baseline_test_02.csv
sample_submission_external_feat_v2_01.csv
scalar_coupling_contributions.csv.zip
structures.csv.zip
structures.zip
test.csv.zip
test_etl.csv
test_feat_eng_v2.csv
train.csv.zip
train_etl.csv
train_feat_eng_v2.csv


In [0]:
!head sample_submission_distance_baseline_test.csv

id,scalar_coupling_constant
4658147,-3.2472324
4658148,91.86721
4658149,10.933329
4658150,107.796974
4658151,1.5662003
4658152,107.1634
4658153,2.08945
4658154,-11.683802
4658155,-10.687826


# Upload to IBM object store

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [0]:
upload_file_cos(credentials_2,'sample_submission_test_3.csv','sample_submission_test_3.csv')

In [0]:
upload_file_cos(credentials_2,'kaggle_model_nn_03.h5','kaggle_model_nn_03.h5')